# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

# Find the ground state using direct minimisation (always using SCF is boring ...)
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.122868e+02     1.591442e+00
 * time: 0.09169983863830566
     1     1.034146e+01     8.903759e-01
 * time: 0.9666829109191895
     2    -1.160161e+01     1.041350e+00
 * time: 1.1044797897338867
     3    -3.379781e+01     7.880703e-01
 * time: 1.2724599838256836
     4    -4.714581e+01     6.419166e-01
 * time: 1.433424949645996
     5    -5.668637e+01     2.768249e-01
 * time: 1.5870769023895264
     6    -5.942955e+01     3.386544e-01
 * time: 1.7011888027191162
     7    -6.058871e+01     1.656004e-01
 * time: 1.8242418766021729
     8    -6.111174e+01     1.023647e-01
 * time: 1.937182903289795
     9    -6.148871e+01     4.077687e-02
 * time: 2.0513648986816406
    10    -6.169761e+01     4.554744e-02
 * time: 2.1729519367218018
    11    -6.188435e+01     2.354294e-02
 * time: 2.288400888442993
    12    -6.198753e+01     1.895439e-02
 * time: 2.4079599380493164
    13    -6.204582e+01     1.455635e-02
 * time: 2.521263837814

In [4]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671076
    AtomicLocal         -18.8557704
    AtomicNonlocal      14.8522656
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485384 
    Xc                  -19.3336825

    total               -62.261666459437
